In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd 
pd.set_option('display.max_columns', None)

import numpy as np
import seaborn as sns
import statsmodels.api as sm
import matplotlib.pyplot as plt
#%matplotlib inline
sns.set(style="whitegrid")
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV 

import warnings 
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
frq_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/frequency_domain_features_train.csv",index_col="uuid")
hr_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/heart_rate_non_linear_features_train.csv",index_col="uuid")
time_domine_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/time_domain_features_train.csv",index_col="uuid")

In [5]:
train_data = pd.concat([frq_data,time_domine_data,hr_data],axis=1)
print(frq_data.shape)
print(hr_data.shape)
print(time_domine_data.shape)
print(train_data.columns)

(369289, 11)
(369289, 6)
(369289, 19)
Index(['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
       'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'SD1', 'SD2',
       'sampen', 'higuci', 'datasetId', 'condition'],
      dtype='object')


In [6]:
train_data.isnull().any()

VLF                  False
VLF_PCT              False
LF                   False
LF_PCT               False
LF_NU                False
HF                   False
HF_PCT               False
HF_NU                False
TP                   False
LF_HF                False
HF_LF                False
MEAN_RR              False
MEDIAN_RR            False
SDRR                 False
RMSSD                False
SDSD                 False
SDRR_RMSSD           False
HR                   False
pNN25                False
pNN50                False
KURT                 False
SKEW                 False
MEAN_REL_RR          False
MEDIAN_REL_RR        False
SDRR_REL_RR          False
RMSSD_REL_RR         False
SDSD_REL_RR          False
SDRR_RMSSD_REL_RR    False
KURT_REL_RR          False
SKEW_REL_RR          False
SD1                  False
SD2                  False
sampen               False
higuci               False
datasetId            False
condition            False
dtype: bool

In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 369289 entries, 89df2855-56eb-4706-a23b-b39363dd605a to 1f2eb52d-4f09-4921-a383-ef1bc6221789
Data columns (total 36 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   VLF                369289 non-null  float64
 1   VLF_PCT            369289 non-null  float64
 2   LF                 369289 non-null  float64
 3   LF_PCT             369289 non-null  float64
 4   LF_NU              369289 non-null  float64
 5   HF                 369289 non-null  float64
 6   HF_PCT             369289 non-null  float64
 7   HF_NU              369289 non-null  float64
 8   TP                 369289 non-null  float64
 9   LF_HF              369289 non-null  float64
 10  HF_LF              369289 non-null  float64
 11  MEAN_RR            369289 non-null  float64
 12  MEDIAN_RR          369289 non-null  float64
 13  SDRR               369289 non-null  float64
 14  RMSSD              369289 non-null  floa

In [8]:
train_data['condition'].value_counts()

no stress        200082
interruption     105150
time pressure     64057
Name: condition, dtype: int64

In [9]:
train_data.describe().T

,count,mean,std,min,25%,50%,75%,max
VLF,369289.0,2199.580170,1815.773422,159.480176,1001.189280,1.667903e+03,2654.121052,12617.977191
VLF_PCT,369289.0,64.289242,16.774844,19.031219,52.909877,6.635024e+01,76.825032,97.738848
LF,369289.0,946.530252,574.171780,90.048557,545.449386,7.827163e+02,1201.432256,3291.548112
LF_PCT,369289.0,34.095182,16.040290,2.165119,22.305936,3.204703e+01,44.647115,77.928847
LF_NU,369289.0,95.566718,4.123365,69.879083,93.645734,9.664314e+01,98.771946,99.987175
HF,369289.0,39.245603,45.398869,0.061783,10.720312,2.484194e+01,45.272368,364.486936
HF_PCT,369289.0,1.615576,1.761073,0.002150,0.346803,1.039513e+00,2.245115,13.095664
HF_NU,369289.0,4.433282,4.123365,0.012825,1.228054,3.356860e+00,6.354266,30.120917
TP,369289.0,3185.356025,1923.227187,377.692795,1828.147788,2.796857e+03,4052.260157,13390.684098
LF_HF,369289.0,115.977200,360.855129,2.319952,14.737458,2.878975e+01,80.429614,7796.443096


In [10]:
train_data["datasetId"].value_counts()

2    369289
Name: datasetId, dtype: int64

In [11]:
# def plot_distribution(data,column_name):
#     fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(5,5))
#     #sns.histplot(data=data,x=column_name,kde=True,ax=ax[0],bins=10)
#     sns.distplot(data[column_name],ax=ax[0],bins=10)
#     sns.boxplot(data=data,y=column_name,orient='v',ax=ax[1])
#     fig.suptitle("{} - {}".format("Distribution for ",column_name))
#     plt.show()

In [12]:
# for column_name in ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
#        'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
#        'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
#        'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
#        'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'SD1', 'SD2',
#        'sampen', 'higuci']:
#     plot_distribution(data,column_name)

In [13]:
# skewness = data.skew(axis=0).to_frame("skewness")
# skewness.style.applymap(lambda x:"background-color:red" if x>3 else ("background-color:orange" if x>1 else ""))


In [14]:
# kurtos = data.kurt(axis=0).to_frame("kurtos")
# kurtos.style.applymap(lambda x:"background-color:red" if x>10 else ("background-color:orange" if x>7 else ""))

In [15]:
train_data = pd.get_dummies(train_data,drop_first=True,columns=["condition"])
train_data.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,HR,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,SD1,SD2,sampen,higuci,datasetId,condition_no stress,condition_time pressure
uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
89df2855-56eb-4706-a23b-b39363dd605a,2661.894136,72.203287,1009.249419,27.375666,98.485263,15.522603,0.421047,1.514737,3686.666157,65.018055,0.015380,885.157845,853.763730,140.972741,15.554505,15.553371,9.063146,69.499952,11.133333,0.533333,-0.856554,0.335218,-0.000203,-0.000179,0.017080,0.007969,0.007969,2.143342,-0.856554,0.335218,11.001565,199.061782,2.139754,1.163485,2,1,0
80c795e4-aa56-4cc0-939c-19634b89cbb2,2314.265450,76.975728,690.113275,22.954139,99.695397,2.108525,0.070133,0.304603,3006.487251,327.296635,0.003055,939.425371,948.357865,81.317742,12.964439,12.964195,6.272369,64.363150,5.600000,0.000000,-0.408190,-0.155286,-0.000059,0.000611,0.013978,0.004769,0.004769,2.930855,-0.408190,-0.155286,9.170129,114.634458,2.174499,1.084711,2,0,0
c2d5d102-967c-487d-88f2-8b005a449f3e,1373.887112,51.152225,1298.222619,48.335104,98.950472,13.769729,0.512671,1.049528,2685.879461,94.280910,0.010607,898.186047,907.006860,84.497236,16.305279,16.305274,5.182201,67.450066,13.066667,0.200000,0.351789,-0.656813,-0.000011,-0.000263,0.018539,0.008716,0.008716,2.127053,0.351789,-0.656813,11.533417,118.939253,2.135350,1.176315,2,0,0
37eabc44-1349-4040-8896-0d113ad4811f,2410.357408,70.180308,1005.981659,29.290305,98.224706,18.181913,0.529387,1.775294,3434.520980,55.328701,0.018074,881.757865,893.460030,90.370537,15.720468,15.720068,5.748591,68.809562,11.800000,0.133333,-0.504947,-0.386138,0.000112,0.000494,0.017761,0.008660,0.008660,2.050988,-0.504947,-0.386138,11.119476,127.318597,2.178341,1.179688,2,1,0
aa777a6a-7aa3-4f6e-aced-70f8691dd2b7,1151.177330,43.918366,1421.782051,54.242160,96.720007,48.215822,1.839473,3.279993,2621.175204,29.487873,0.033912,809.625331,811.184865,62.766242,19.213819,19.213657,3.266724,74.565728,20.200000,0.200000,-0.548408,-0.154252,-0.000100,-0.002736,0.023715,0.013055,0.013055,1.816544,-0.548408,-0.154252,13.590641,87.718281,2.221121,1.249612,2,1,0


In [16]:
frq_data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/frequency_domain_features_test.csv",index_col="uuid")
hr_data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/heart_rate_non_linear_features_test.csv",index_col="uuid")
time_domine_data_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/time_domain_features_test.csv",index_col="uuid")

test_data = pd.concat([frq_data_test,time_domine_data_test,hr_data_test],axis=1)

In [17]:
print(frq_data_test.shape)
print(hr_data_test.shape)
print(time_domine_data_test.shape)
print(test_data.columns)
print(test_data.isnull().any())
print(test_data.info())

(41033, 11)
(41033, 6)
(41033, 18)
Index(['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDSD_REL_RR',
       'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'SD1', 'SD2',
       'sampen', 'higuci', 'datasetId', 'condition'],
      dtype='object')
VLF                  False
VLF_PCT              False
LF                   False
LF_PCT               False
LF_NU                False
HF                   False
HF_PCT               False
HF_NU                False
TP                   False
LF_HF                False
HF_LF                False
MEAN_RR              False
MEDIAN_RR            False
SDRR                 False
RMSSD                False
SDSD                 False
SDRR_RMSSD           False
pNN25                False
pNN50                False
KURT     

In [18]:
test_data['condition'].value_counts()

no stress        22158
interruption     11782
time pressure     7093
Name: condition, dtype: int64

In [19]:
test_data.describe().T

,count,mean,std,min,25%,50%,75%,max
VLF,41033.0,2.196457e+03,1815.262687,160.869730,994.970318,1.665050e+03,2657.910801,12427.177612
VLF_PCT,41033.0,6.422104e+01,16.838147,19.301549,52.721914,6.624026e+01,76.845957,97.739588
LF,41033.0,9.488518e+02,577.413541,92.663980,544.688588,7.834525e+02,1210.276983,3290.093757
LF_PCT,41033.0,3.416810e+01,16.105461,2.163917,22.299698,3.217173e+01,44.754617,77.522361
LF_NU,41033.0,9.558661e+01,4.102133,70.682584,93.659224,9.665809e+01,98.794732,99.987029
HF,41033.0,3.901211e+01,44.994281,0.062809,10.551975,2.465485e+01,45.004948,360.877726
HF_PCT,41033.0,1.610860e+00,1.750709,0.002211,0.338246,1.034102e+00,2.237060,13.087775
HF_NU,41033.0,4.413387e+00,4.102133,0.012971,1.205268,3.341912e+00,6.340776,29.317416
TP,41033.0,3.184321e+03,1923.544936,378.053805,1820.982721,2.799393e+03,4055.490025,13249.507794
LF_HF,41033.0,1.171816e+02,368.039153,2.410942,14.770940,2.892299e+01,81.969112,7708.369846


In [20]:
test_data["datasetId"].value_counts()

2    41033
Name: datasetId, dtype: int64

In [21]:
test_data = pd.get_dummies(test_data,drop_first=True,columns=["condition"])
test_data.head()

,VLF,VLF_PCT,LF,LF_PCT,LF_NU,HF,HF_PCT,HF_NU,TP,LF_HF,HF_LF,MEAN_RR,MEDIAN_RR,SDRR,RMSSD,SDSD,SDRR_RMSSD,pNN25,pNN50,KURT,SKEW,MEAN_REL_RR,MEDIAN_REL_RR,SDRR_REL_RR,RMSSD_REL_RR,SDSD_REL_RR,SDRR_RMSSD_REL_RR,KURT_REL_RR,SKEW_REL_RR,SD1,SD2,sampen,higuci,datasetId,condition_no stress,condition_time pressure
uuid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
62b75db5-bc40-4c8f-9166-daf0efcab4c2,1868.532278,76.511189,570.643114,23.366245,99.478197,2.993254,0.122565,0.521803,2442.168645,190.643094,0.005245,934.665288,939.03173,82.139495,11.801781,11.801772,6.959924,3.933333,0.133333,-0.680262,-0.233075,0.000016,0.000288,0.012933,0.004578,0.004578,2.825038,-0.680262,-0.233075,8.347898,115.862444,2.209659,1.100715,2,0,1
a99549ad-3eb6-4413-bc90-9053e7f7e684,568.742845,26.301350,1553.971621,71.862973,97.509212,39.694850,1.835677,2.490788,2162.409316,39.147940,0.025544,817.062380,816.33879,55.492332,20.558810,20.558768,2.699200,24.600000,0.533333,-0.034454,-0.051689,0.000040,-0.002749,0.025148,0.013921,0.013921,1.806517,-0.034454,-0.051689,14.542096,77.118903,2.186132,1.290615,2,1,0
cb573d3a-c767-4556-b32e-ad8c08ded214,2101.871207,75.836461,655.175895,23.639042,97.829386,14.536877,0.524497,2.170614,2771.583978,45.069921,0.022188,876.762022,894.19889,88.690820,13.853737,13.853730,6.401942,7.066667,0.533333,-0.206953,-0.589940,0.000010,-0.000172,0.015533,0.008149,0.008149,1.906180,-0.206953,-0.589940,9.799336,125.044377,2.051571,1.226663,2,1,0
47a0c6de-2aef-4ac3-997d-252fa6fd07f1,5757.544433,90.562305,592.913021,9.326123,98.817806,7.093235,0.111572,1.182194,6357.550689,83.588517,0.011963,1038.640693,998.91429,213.725850,16.457194,16.454801,12.986774,10.800000,1.866667,-0.820407,0.487198,-0.000238,-0.000464,0.016882,0.007587,0.007587,2.225116,-0.820407,0.487198,11.639185,302.029812,2.080910,1.085143,2,0,1
de3fd54f-c74e-4fe8-bf2a-7a127f68b312,964.696325,70.256575,374.939530,27.305968,91.805057,33.468834,2.437457,8.194943,1373.104689,11.202647,0.089265,774.548508,778.90508,51.577855,10.273114,10.273049,5.020664,2.000000,0.066667,1.738453,-0.005082,0.000054,-0.000006,0.013479,0.007811,0.007811,1.725606,1.738453,-0.005082,7.266567,72.579248,2.068728,1.252547,2,0,0


In [22]:
x_columns = ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT',
       'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR',
       'RMSSD', 'SDSD', 'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW',
       'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR',
       'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'SD1',
       'SD2', 'sampen', 'higuci', 'condition_no stress', 'condition_time pressure']

y_columns = ['HR']

X = train_data[x_columns]
y = train_data[y_columns]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)


In [ ]:
parameters = {'max_features':[2,3,4,5,6], 
              'n_estimators':[100],
              'max_depth':[3,6,9],
               'min_samples_split':[5, 10, 15, 100],
                'min_samples_leaf':[ 5, 10]               
             }
rfg = RandomForestRegressor(random_state=101)            
model = GridSearchCV(rfg,parameters,cv=10)          
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
print(mean_absolute_error(y_test,y_predict))
print("Best hyperparams--",model.best_params_)